In [1]:
import pandas as pd

metadata = pd.read_csv("./datasets/movies_metadata.csv")
links = pd.read_csv("./datasets/links.csv")

C:\Users\USER\AppData\Local\Temp\ipykernel_9868\3087031659.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("./datasets/movies_metadata.csv")


convert imdb_id to int

In [2]:
metadata['imdb_id'] = metadata['imdb_id'].str.replace('tt', '')
metadata['imdb_id'] = metadata['imdb_id'].astype('Int64')

metadata = pd.merge(metadata, links, left_on='imdb_id', right_on='imdbId', how='inner')
del links

metadata.shape

(45383, 27)

In [4]:
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'movieId', 'imdbId', 'tmdbId'],
      dtype='object')

pick the movie which has more than 10 ratings to reduce dimentionality for precessing

In [5]:
metadata = metadata[metadata['vote_count'] > 10]
metadata.shape

(21745, 27)

In [6]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709,862.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497,8844.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,3,113228,15602.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,4,114885,31357.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5,113041,11862.0


use ast library to extract data grom json and dict objects

In [6]:
import ast

metadata['genres'] = metadata['genres'].apply(lambda x: ', '.join([i['name'] for i in ast.literal_eval(x)] if isinstance(x, str) else ''))
metadata['production_companies'] = metadata['production_companies'].apply(lambda x: ', '.join([i['name'] for i in ast.literal_eval(x)] if isinstance(x, str) else ''))
metadata['production_countries'] = metadata['production_countries'].apply(lambda x: ', '.join([i['name'] for i in ast.literal_eval(x)] if isinstance(x, str) else ''))

metadata[['genres', 'production_companies', 'production_countries']]


,genres,production_companies,production_countries
0,"Animation, Comedy, Family",Pixar Animation Studios,United States of America
1,"Adventure, Fantasy, Family","TriStar Pictures, Teitler Film, Interscope Com...",United States of America
2,"Romance, Comedy","Warner Bros., Lancaster Gate",United States of America
3,"Comedy, Drama, Romance",Twentieth Century Fox Film Corporation,United States of America
4,Comedy,"Sandollar Productions, Touchstone Pictures",United States of America
...,...,...,...
45360,"Comedy, Fantasy, Science Fiction",Star Film Company,France
45363,"Fantasy, Comedy",Star Film Company,France
45367,"Fantasy, Action, Thriller",Star Film Company,France
45370,"Crime, Drama, Thriller","Mad Films, Third Eye Pictures",India


In [8]:
import numpy as np

metadata['order'] = np.log(metadata['vote_count']) * (metadata['vote_average'] - 5)

In [9]:
# save to csv
metadata.to_csv('app/exports/metadata.csv', index=False)